# Preprocesado de datos

El preprocesado de datos es una fase indispensable para el correndo aprendizaje por partes de los algoritmos de Deeplearning. Se ha demostrado empíricamente que una correcta preparación y normalización de los datos permiten hallar soluciones más cercanas a la optima que con datos no procesados.

Es importante tener en cuenta que no existe una metodología de preprocesado única, y que es necesario adaptarse al tipo de dato que estamos tratando. Para este proyecto, además, existe una dificultad adicional, y es la existencia de diferentes procedencias para los datos, pues en total se dispone de 5 datasets distintos, cada uno recopilado con diferentes metodologías e instrumentación. Por tanto, será clave adaptarse a cada uno de los destinos, y realizar la partición final de forma estratificada para evitar sesgos que perturben el resutado.

Más adelante, profundizaremos en este aspecto, pero en primer lugar, debemos leer cada uno de los conjuntos de datos disponibles, y examinar de cuántos elementos disponemos en cada uno, para establecer la proporción de entrenamiento-test oportuna.

In [32]:
# Librerías utilizadas por el script
import os
import cv2
import zipfile

import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset disponibles

Haciendo uso de los recursos disponibles públicamente, se han tomado los siguientes datasets para realizar el experimento:
- ISIC: es el mayor conjuntos de datos cutáneos disponible en abierto, y contiene imágenes de todo tipo de pieles y procedencias, pero con especial énfasis en las personas de origen europeo y americano.
- ASAN: Conjunto de datos provenientes del hospital con este mismo nombre, con lesiones en personas de origen asiático.
- PAD UFES 20: conjunto de datos de lesiones variadas de pacientes latinoamericanos.
- PH2: banco de datos de pacientes brasileños con lesiones potencialmente cancerosas.
- Severance: base de datos con lesiones cutáneas en población asiática, con contenido tanto benigno como cancerígeno.

Para unificar la notación de los datos, se creará un código para la notación de cada una de las clases que permitan un procesamiento común de todos los datos sin depender del origen de este. Para ello, se creará un nuevo archivo .csv donde se anotará el path de la imagen, su clase asociada, y el tipo general de la misma (beningna o maligna). La metainformación asociada, de momento, quedará relegado a un segundo plano hasta el estudio estadístico de los datos.

In [33]:
# Directorios de cada dataset

ISIC_PATH = "datasets/ISIC"
ASAN_PATH = "datasets/ASAN"
PAD_UFES_PATH = "datasets/PAD_UFES_20"
PH2_PATH = "datasets/PH2"
SEVERANCE = "datasets/Severance"

In [8]:
# Funciones comunes para la lectura y muestra de datos

'''
This function receives a string with the filename of the image to read,
and a flag indicating if we want to read it in color/RGB (flagColor=1) or gray level (flagColor=0)

Example of use:
im1=readIm(get_image('apple.jpg'),0)
'''


def readIm(filename, flagColor=1):
    # cv2 reads BGR format
    im = cv2.imread(filename)
    # change to  RGB and return the image
    if (flagColor):
        return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # change from BGR to grayscale instead if flag is 0
    return cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)


'''
This function receives an array of arbitrary real numbers (that could include even negative values),
and returns an 'image' in the range [0,1].
flag_GLOBAL allows the user to normalize the whole image (including all channels) or to normalize
each channel/band independently.
'''


def rangeDisplay01(im, flag_GLOBAL=True):
    im = im.astype(float)
    if flag_GLOBAL:
        im = (im - im.min()) / (im.max() - im.min())
    else:
        # bands normalization
        for band in range(im.shape[2]):
            im[:, :, band] = (im[:, :, band] - im[:, :, band].min()) / (im[:, :, band].max() - im[:, :, band].min())
            # Note: remember that, for plt.imshow with RGB data, the valid range is [0..1] for floats and [0..255] for integers.
    return im


"""
Función para mostrar imágenes en pantalla en color y blanco negro. Permite realizar
aumento sobre las mismas para apreciar un mayor detalle.

Entrada:
    im: imagen leída en formato ndarray
    title: nombre que recibe el marco en pantalla
    factor: factor de aumento de la image, "zoom"
"""


def displayIm(im, title='Result', factor=2):
    # First normalize range
    max = np.max(im)
    min = np.min(im)
    if min < 0 or max > 255:
        im = rangeDisplay01(im, flag_GLOBAL=True)
    if len(im.shape) == 3:
        # im es tribanda
        plt.imshow(im, cmap='jet')
    else:
        # im es monobanda
        plt.imshow(im, cmap='gray')
    figure_size = plt.gcf().get_size_inches()
    plt.gcf().set_size_inches(factor * figure_size)
    plt.title(title)
    plt.xticks([]), plt.yticks([])  # eliminamos numeración
    plt.show()

### ISIC Skin Dataset

Se trata del dataset de mayor tamaño del conjunto. Contiene 31 clases identificadas, tanto de lesiones benignas y malignas de la piel. En total, se dispone de 53738, las cuales ya han sido filtradas para asegurarse de que no exista redundancia por las herramientas online de la galería ISIC: https://gallery.isic-archive.com/

In [36]:
def extractISIC(path: str):
    # Obtener una lista de todos los archivos ZIP en la carpeta especificada
    archivos_zip = [f for f in os.listdir(path) if f.lower().endswith('.zip')]

    # Iterar sobre cada archivo ZIP
    for archivo_zip in archivos_zip:
        ruta_archivo_zip = os.path.join(path, archivo_zip)
        carpeta_salida = os.path.splitext(ruta_archivo_zip)[0]  # Eliminar la extensión .zip

        # Comprobar si la carpeta de salida ya existe
        if not os.path.exists(carpeta_salida):
            os.makedirs(carpeta_salida)  # Crear la carpeta de salida

            # Extraer el contenido del archivo ZIP en la carpeta de salida
            with zipfile.ZipFile(ruta_archivo_zip, 'r') as zip_ref:
                zip_ref.extractall(carpeta_salida)
            print(f"Extraído {archivo_zip} en {carpeta_salida}")
        else:
            print(f"Omitido {archivo_zip}. {carpeta_salida} ya existe.")

In [37]:

# Accedemos al directorio de ISIC
print(os.getcwd())

# Extraemos cada zip, en caso de que no exista
extractISIC(ISIC_PATH)

print(os.getcwd())



C:\Users\Cris1\Documents\TFG
Extraído Acrochordon.zip en datasets/ISIC\Acrochordon
Extraído Acticnic keratosis.zip en datasets/ISIC\Acticnic keratosis
Extraído AIMP.zip en datasets/ISIC\AIMP
Extraído Angiofroma o fibreus papile.zip en datasets/ISIC\Angiofroma o fibreus papile
Extraído Angiokeratoma.zip en datasets/ISIC\Angiokeratoma
Extraído Angioma.zip en datasets/ISIC\Angioma
Extraído Atypical melatocynic proliferation.zip en datasets/ISIC\Atypical melatocynic proliferation
Extraído Atypical spitz tumor.zip en datasets/ISIC\Atypical spitz tumor
Extraído Basal Cell Carcinoma.zip en datasets/ISIC\Basal Cell Carcinoma
Extraído Cafe au lait macule.zip en datasets/ISIC\Cafe au lait macule
Extraído Clear cell acarthoma.zip en datasets/ISIC\Clear cell acarthoma
Extraído Dermatofibroma.zip en datasets/ISIC\Dermatofibroma


KeyboardInterrupt: 